In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import laplace #Image, 2D Data Filter

In [7]:
# 3x3x3 어레이 초기화
default_array = np.zeros((3, 3, 3))
fill_array2 = np.full((3, 3, 3), 6)

# 100x100x100 격자 생성
x = np.linspace(0, 99, 100)
y = np.linspace(0, 99, 100)
z = np.linspace(0, 99, 100)

# 3D 메쉬 그리드
X2, Y2 = np.meshgrid(x, y, indexing='ij')
X3, Y3, Z3 = np.meshgrid(x, y, z, indexing='ij')

# 스칼라 필드 정의
f = x**2                              # 1D 스칼라 필드
f2 = X2**2 + Y2**2                      # 2D 스칼라 필드
f3 = X3**2 + Y3**2 + Z3**2               # 3D 스칼라 필드

# Gradient 계산
dfdx = np.gradient(f, x)             # 1D gradient
dfdx2, dfdy2 = np.gradient(f2, x, y, edge_order=2)  # 2D gradient
dfdx3, dfdy3, dfdz3 = np.gradient(f3, x, y, z, edge_order=2)  # 3D gradient

# 결과 확인
print("∂f/∂x at x[10]:", dfdx[10])
print("∂f2/∂x at x=10:", dfdx2[10, 0])  # 3D 배열이라 슬라이싱 필요
print("∂f3/∂x at x=10:", dfdx3[10, 0, 0])
print("∂f2/∂x at x=10:", dfdy2[0, 10])  # 3D 배열이라 슬라이싱 필요
print("∂f3/∂x at x=10:", dfdy3[0, 10, 0])

# 정의된 배열로 gradient 수행
grad_default = np.gradient(fill_array2)  # 또는 default_array
print("gradient of fill_array2 shape:", [g.shape for g in grad_default])


∂f/∂x at x[10]: 20.0
∂f2/∂x at x=10: 20.0
∂f3/∂x at x=10: 20.0
∂f2/∂x at x=10: 20.0
∂f3/∂x at x=10: 20.0
gradient of fill_array2 shape: [(3, 3, 3), (3, 3, 3), (3, 3, 3)]


In [3]:
def compute_3d_gradient(f, dx, dy, dz):
    """
    f : 3D 스칼라 필드 (numpy array, shape = [Nx, Ny, Nz])
    dx, dy, dz : 각 방향의 격자 간격
    return : gradient (gx, gy, gz) 각 방향의 미분 결과
    """
    gx = np.zeros_like(f)
    gy = np.zeros_like(f)
    gz = np.zeros_like(f)

    # 중심 차분 내부
    gx[1:-1, :, :] = (f[2:, :, :] - f[:-2, :, :]) / (2 * dx)
    gy[:, 1:-1, :] = (f[:, 2:, :] - f[:, :-2, :]) / (2 * dy)
    gz[:, :, 1:-1] = (f[:, :, 2:] - f[:, :, :-2]) / (2 * dz)

    # 전/후방 차분 (경계 처리)
    gx[0, :, :] = (f[1, :, :] - f[0, :, :]) / dx
    gx[-1, :, :] = (f[-1, :, :] - f[-2, :, :]) / dx

    gy[:, 0, :] = (f[:, 1, :] - f[:, 0, :]) / dy
    gy[:, -1, :] = (f[:, -1, :] - f[:, -2, :]) / dy

    gz[:, :, 0] = (f[:, :, 1] - f[:, :, 0]) / dz
    gz[:, :, -1] = (f[:, :, -1] - f[:, :, -2]) / dz

    return gx, gy, gz
# 격자 크기
Nx, Ny, Nz = 100, 100, 100
dx = dy = dz = 1

# 격자 생성
x = np.linspace(0, dx*(Nx-1), Nx)
y = np.linspace(0, dy*(Ny-1), Ny)
z = np.linspace(0, dz*(Nz-1), Nz)
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

# 스칼라 필드 정의: f(x, y, z) = x^2 + y^2 + z^2
f = X**2 + Y**2 + Z**2

# gradient 계산
gx, gy, gz = compute_3d_gradient(f, dx, dy, dz)

# 특정 포인트의 결과 확인
print("∂f/∂x at center:", gx[10, 0, 0])  # ≈ 2x
print("∂f/∂y at center:", gy[0, 10, 0])  # ≈ 2y
print("∂f/∂z at center:", gz[0, 0, 10])  # ≈ 2z dd


∂f/∂x at center: 20.0
∂f/∂y at center: 20.0
∂f/∂z at center: 20.0


In [5]:
def laplace_3d(f, dx, dy, dz):
    lap = np.zeros_like(f)
    lap[1:-1,1:-1,1:-1] = (
        (f[2:,1:-1,1:-1] - 2*f[1:-1,1:-1,1:-1] + f[:-2,1:-1,1:-1]) / dx**2 +
        (f[1:-1,2:,1:-1] - 2*f[1:-1,1:-1,1:-1] + f[1:-1,:-2,1:-1]) / dy**2 +
        (f[1:-1,1:-1,2:] - 2*f[1:-1,1:-1,1:-1] + f[1:-1,1:-1,:-2]) / dz**2
    )
    return lap

In [16]:
x = np.linspace(0, 99, 100)
y = np.linspace(0, 99, 100)
z = np.linspace(0, 99, 100)
dx = x[1] - x[0]
dy = y[1] - y[0]
dz = z[1] - z[0]
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
f = X**2 + Y**2 + Z**2
f2 = X**3 + Y**3 + Z**3
lap_f = laplace_3d(f, dx, dy, dz)
lap_f2 = laplace_3d(f2, dx, dy, dz)
#print(lap_f)
print("∂^2f/∂x^2 + ∂^2f/∂y^2 + ∂^2f/∂z^2 :", lap_f[1, 1, 1])  # ≈ 6
print("∂^2f/∂x^2 + ∂^2f/∂y^2 + ∂^2f/∂z^2 :", lap_f2[10, 1, 1])  # ≈ 6x + 6y + 6z

∂^2f/∂x^2 + ∂^2f/∂y^2 + ∂^2f/∂z^2 : 6.0
∂^2f/∂x^2 + ∂^2f/∂y^2 + ∂^2f/∂z^2 : 72.0


In [17]:
def partial_derivative(f, axis, dx):
    """
    3D 스칼라 필드 f 에 대해 x/y/z 중 한 방향으로 1차 편미분을 수행합니다.
    
    Parameters:
    - f    : (Nx, Ny, Nz) 형태의 numpy 배열
    - axis : 'x', 'y', 'z' 중 하나 (편미분 방향)
    - dx   : 해당 축의 격자 간격

    Returns:
    - df_daxis : 편미분 결과 (f와 동일한 shape)
    """
    df = np.zeros_like(f)

    if axis == 'x':
        df[1:-1,:,:] = (f[2:,:,:] - f[:-2,:,:]) / (2*dx)
        df[0,:,:]    = (f[1,:,:] - f[0,:,:]) / dx
        df[-1,:,:]   = (f[-1,:,:] - f[-2,:,:]) / dx
    elif axis == 'y':
        df[:,1:-1,:] = (f[:,2:,:] - f[:,:-2,:]) / (2*dx)
        df[:,0,:]    = (f[:,1,:] - f[:,0,:]) / dx
        df[:,-1,:]   = (f[:,-1,:] - f[:,-2,:]) / dx
    elif axis == 'z':
        df[:,:,1:-1] = (f[:,:,2:] - f[:,:,:-2]) / (2*dx)
        df[:,:,0]    = (f[:,:,1] - f[:,:,0]) / dx
        df[:,:,-1]   = (f[:,:,-1] - f[:,:,-2]) / dx
    else:
        raise ValueError("axis는 'x', 'y', 'z' 중 하나여야 합니다.")
    
    return df

In [18]:
# 격자 생성
Domain = 100
nx, ny, nz = 100, 100, 100
x = np.linspace(0, Domain-1, nx)
y = np.linspace(0, Domain-1, ny)
z = np.linspace(0, Domain-1, nz)
dx = x[1] - x[0]

# Meshgrid
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

# f(x,y,z) = x² + y² + z²
f = X**2 + Y**2 + Z**2

# x partial_derivative
dfdx = partial_derivative(f, 'x', dx)
# y partial_derivative
dfdy = partial_derivative(f, 'y', dx)
# z partial_derivative
dfdz = partial_derivative(f, 'z', dx)

print("∂f/∂x :", dfdx[1, 0, 0])  # ≈ 2
print("∂f/∂y :", dfdy[0, 1, 0])  # ≈ 2
print("∂f/∂z :", dfdz[0, 0, 1])  # ≈ 2

∂f/∂x : 2.0
∂f/∂y : 2.0
∂f/∂z : 2.0


In [ ]:
# 격자 생성
Domain = 100
nx, ny, nz = 100, 100, 100
Density = 1.225  # kg/m^3 ( 15 Celsius )
Viscosity = 1.81e-5 # Air Viscosity

In [ ]:
fx = np.zeros((nx, ny, nz))
fy = np.zeros((nx, ny, nz))
fz = np.full((nx, ny, nz), -Density * 9.81)  # 중력 가속도 방향

In [ ]:
Velocity_eq =
pressure_eq = 
Stress_eq = 
external_force_eq = 

In [ ]:
# TODO : 
# 1. electomanetic simul
# 2. thermo simul
# 3. Young's Modulus simul

![nn](Navier.png)

In [ ]:
def compute_volume_lift(u, v, w, p, rho, mu, fx, fy, fz, dx, dy, dz, dt):
    # 시간 도함수
    du_dt = (u_new - u_old) / dt
    dv_dt = (v_new - v_old) / dt
    dw_dt = (w_new - w_old) / dt

    # 대류항
    conv_u = u * gradient_3d(u, dx, dy, dz)[0] + v * gradient_3d(u, dx, dy, dz)[1] + w * gradient_3d(u, dx, dy, dz)[2]
    conv_v = u * gradient_3d(v, dx, dy, dz)[0] + v * gradient_3d(v, dx, dy, dz)[1] + w * gradient_3d(v, dx, dy, dz)[2]
    conv_w = u * gradient_3d(w, dx, dy, dz)[0] + v * gradient_3d(w, dx, dy, dz)[1] + w * gradient_3d(w, dx, dy, dz)[2]

    # 압력 기울기
    dp_dx, dp_dy, dp_dz = gradient_3d(p, dx, dy, dz)

    # 점성항
    lap_u = laplace_3d(u, dx, dy, dz)
    lap_v = laplace_3d(v, dx, dy, dz)
    lap_w = laplace_3d(w, dx, dy, dz)

    # 양력 방향 성분 (예: z방향 기준)
    integrand = -rho * (dw_dt + conv_w) - dp_dz + mu * lap_w - fz
    Lz = np.sum(integrand) * dx * dy * dz
    return Lz
